In [1]:
import psycopg2
import pandas as pd

## Ссылки:
- `Read sql query from DB to pandas` - https://pandas.pydata.org/docs/reference/api/pandas.read_sql_query.html
- `Must read for SQL` - https://habr.com/ru/post/564390/ 

In [2]:
username = 'irina'
password = 'irina_google'
host = '77.244.65.15'

In [3]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [4]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [5]:
conn = psycopg2.connect(
        dbname='dwh',
        user=username,
        password=password,
        host=host,
        port=5432)
cursor = conn.cursor()

In [19]:
# 123
"""
123
"""
a = 'abc'
b = "abc"
c = """
abc
"""
print(a)
print(b)
print(c)

abc
abc

abc



In [11]:
query = '''
    select 
        id
        , link 
        , title
        , city 
        , bank_name
        , score 
        , status 
        , username 
        , create_dt 
        , comments
    from home.dt_banki_responses
    order by id desc 
    limit 10
'''
cursor.execute(query)
result = cursor.fetchall()

In [15]:
for i in result:
    print(i)
    print('----')

(10574978, 'https://www.banki.ru/services/responses/bank/response/10574978/', 'Отзыв об обслуживании', 'г. Йошкар-Ола (Республика Марий Эл)', 'Газпромбанк', 3, 'Проверяется', '89024326903', datetime.datetime(2021, 10, 18, 18, 45, tzinfo=psycopg2.tz.FixedOffsetTimezone(offset=0, name=None)), 0)
----
(10574975, 'https://www.banki.ru/services/responses/bank/response/10574975/', 'Кредитная карта 110 дней', 'г. Москва', 'Райффайзенбанк', 1, 'Проверяется', 'Sergo1984', datetime.datetime(2021, 10, 18, 18, 36, tzinfo=psycopg2.tz.FixedOffsetTimezone(offset=0, name=None)), 0)
----
(10574968, 'https://www.banki.ru/services/responses/bank/response/10574968/', 'Прошу отозвать персональные данные.', 'г. Челябинск (Челябинская область)', 'Райффайзенбанк', 1, 'Проверяется', 'Uralgeo', datetime.datetime(2021, 10, 18, 18, 22, tzinfo=psycopg2.tz.FixedOffsetTimezone(offset=0, name=None)), 0)
----
(10574967, 'https://www.banki.ru/services/responses/bank/response/10574967/', 'Альфа банк и Альфа страхование 

In [7]:
def custom_read_from_sql(query, n):
    df = pd.pandas.read_sql_query(query, conn)
    return df.head(n)

In [8]:
q = """
    select 
        *
    from 
        home.dt_banki_responses 
    where 
        date(create_dt) >= '2021-01-01'
"""
custom_read_from_sql(q, 5)

,id,link,title,city,bank_name,score,status,username,create_dt,comments,content,bank_answer,bank_answer_date,admin_answer,admin_answer_date,parse_dt
0,10528581,https://www.banki.ru/services/responses/bank/r...,Адекватный банк,г. Москва,Хоум Кредит Банк,4.0,Не засчитана,house5,2021-07-09 19:11:00+00:00,1,Являюсь клиентом банка около полугода. Банк пр...,"Добрый день.Спасибо за отзыв. Приятно, что вза...",2021-07-13 10:11:00+00:00,".. Мы не получили ответ от автора, оценку не з...",2021-09-21 14:17:00+00:00,2021-09-21
1,10528512,https://www.banki.ru/services/responses/bank/r...,Прогресс на лицо,г. Ульяновск (Ульяновская область),Хоум Кредит Банк,4.0,Не засчитана,Alenka030591,2021-07-09 17:50:00+00:00,1,Знакомство с этим банком произошло давным давн...,"Добрый деньПрекрасно, что наш Банк запомнился ...",2021-07-12 19:15:00+00:00,".. Ответ от автора не получен, оценку не засчи...",2021-09-18 16:37:00+00:00,2021-09-21
2,10528240,https://www.banki.ru/services/responses/bank/r...,Лучший банк 2021,г. Иркутск (Иркутская область),Хоум Кредит Банк,5.0,Без статуса,Sandy333,2021-07-09 12:57:00+00:00,5,"Недавно стал клиентом банка, порадовало следую...","Добрый день.Спасибо за отзыв. Приятно, что вза...",2021-07-12 16:02:00+00:00,Спасибо за отзыв.отзывчивые сотрудники колцент...,2021-07-16 17:55:00+00:00,2021-09-21
3,10528190,https://www.banki.ru/services/responses/bank/r...,Хорошая дебетовка и накопительный счет,г. Москва,Хоум Кредит Банк,4.0,Не засчитана,strau,2021-07-09 12:03:00+00:00,1,"Обычно писал в НР о проблемах с банками, разба...","Добрый день.Спасибо за отзыв. Приятно, что вза...",2021-07-12 15:52:00+00:00,Спасибо за отзыв Вы описали заявленные условия...,2021-07-16 11:24:00+00:00,2021-09-21
4,10528164,https://www.banki.ru/services/responses/bank/r...,Платеж не проведен,г. Нижний Новгород (Нижегородская область),Хоум Кредит Банк,2.0,Не засчитана,t*******@gmail.com,2021-07-09 11:35:00+00:00,0,Стал клиентом данного банка потому что увидел ...,"Добрый день.К сожалению, вы не предоставили на...",2021-07-20 14:22:00+00:00,А вот работой Чатподдержки в мобильном приложе...,2021-07-16 09:44:00+00:00,2021-09-21


In [10]:
q = """
    select 
        bank_name
        , count(*) as count
        , round(avg(coalesce(score, 0)), 2) as avg_score
        , round(median(coalesce(score, 0)), 2) as median
        , sum(iif(coalesce(score, 0)=0, 1, 0)) as count_0
        , sum(iif(score=1, 1, 0)) as count_1
        , sum(iif(score=2, 1, 0)) as count_2
        , sum(iif(score=3, 1, 0)) as count_3
        , sum(iif(score=4, 1, 0)) as count_4
        , sum(iif(score=5, 1, 0)) as count_5
    from 
        home.dt_banki_responses 
    where 
        date(create_dt) >= '2021-01-01'
    group by bank_name 
    order by count desc
"""
custom_read_from_sql(q, 15)

,bank_name,count,avg_score,median,count_0,count_1,count_2,count_3,count_4,count_5
0,Тинькофф Банк,13840,4.51,5.0,297,1076,192,86,277,11912
1,ВТБ,9998,2.07,1.0,1338,4937,786,190,125,2622
2,Хоум Кредит Банк,5404,3.53,5.0,281,1261,221,100,610,2931
3,Альфа-Банк,5203,1.19,1.0,786,3581,435,123,30,248
4,Банк Открытие,2981,1.84,1.0,379,1628,265,74,61,574
5,Газпромбанк,2860,1.79,1.0,266,1700,269,87,65,473
6,Почта Банк,1225,1.40,1.0,160,829,81,19,7,129
7,МТС Банк,1225,2.93,3.0,121,428,57,10,32,577
8,Райффайзенбанк,1120,1.43,1.0,183,680,96,29,16,116
9,Точка,679,4.63,5.0,18,24,6,7,32,592


In [59]:
q = """
with prepare_table as
(
    select 
        city as raw_city
        , substring(city, '\((.*?)\)') as area
        --, split_part(city, '(',1)
        --, substr(split_part(city, '(',1), 4, length(split_part(city, '(',1))-1) as raw_city
        , substr(split_part(city, '(',1), 4) as city_cln
        , *
    from home.dt_banki_responses
    where 
        date(create_dt) >= '2021-01-01'
)
select 
    city_cln
    , count(*) as count
    , round(avg(coalesce(score, 0)), 2) as avg_score
    , round(median(coalesce(score, 0)), 2) as median
    , sum(iif(coalesce(score, 0)=0, 1, 0)) as count_0
    , sum(iif(score=1, 1, 0)) as count_1
    , sum(iif(score=2, 1, 0)) as count_2
    , sum(iif(score=3, 1, 0)) as count_3
    , sum(iif(score=4, 1, 0)) as count_4
    , sum(iif(score=5, 1, 0)) as count_5
from prepare_table
group by city_cln
order by count desc
"""
custom_read_from_sql(q, 10)

,city_cln,count,avg_score,median,count_0,count_1,count_2,count_3,count_4,count_5
0,Москва,16834,2.56,1.0,1689,7121,1020,323,395,6286
1,Санкт-Петербург,3893,2.38,1.0,375,1797,283,79,127,1232
2,Екатеринбург,968,2.96,3.0,70,347,56,17,37,441
3,Новосибирск,929,2.64,2.0,103,359,62,12,32,361
4,Казань,813,3.25,5.0,57,249,29,16,20,442
5,Краснодар,783,3.25,5.0,48,245,39,8,21,422
6,Уфа,716,3.56,5.0,50,161,37,4,21,443
7,Ростов-на-Дону,685,3.36,5.0,34,210,26,7,18,390
8,Нижний Новгород,668,2.93,2.0,50,242,43,10,17,306
9,Воронеж,641,3.24,5.0,49,196,23,7,18,348


In [61]:
q = """
with prepare_table as
(
    select 
        city as raw_city
        , substring(city, '\((.*?)\)') as area
        --, split_part(city, '(',1)
        --, substr(split_part(city, '(',1), 4, length(split_part(city, '(',1))-1) as raw_city
        , substr(split_part(city, '(',1), 4) as city_cln
        , *
    from home.dt_banki_responses
    where 
        date(create_dt) >= '2021-01-01'
)
select 
    area
    , count(*) as count
    , round(avg(coalesce(score, 0)), 2) as avg_score
    , round(median(coalesce(score, 0)), 2) as median
    , sum(iif(coalesce(score, 0)=0, 1, 0)) as count_0
    , sum(iif(score=1, 1, 0)) as count_1
    , sum(iif(score=2, 1, 0)) as count_2
    , sum(iif(score=3, 1, 0)) as count_3
    , sum(iif(score=4, 1, 0)) as count_4
    , sum(iif(score=5, 1, 0)) as count_5
from prepare_table
where area is not null
group by area
order by count desc
"""
custom_read_from_sql(q, 10)

,area,count,avg_score,median,count_0,count_1,count_2,count_3,count_4,count_5
0,Краснодарский край,1383,3.34,5.0,82,413,61,12,23,792
1,Свердловская область,1255,3.08,4.0,93,416,65,21,47,613
2,Республика Татарстан,1152,3.38,5.0,78,321,39,23,28,663
3,Республика Башкортостан,998,3.74,5.0,59,200,42,6,27,664
4,Ростовская область,990,3.55,5.0,59,251,34,7,24,615
5,Новосибирская область,977,2.64,1.0,111,378,62,12,32,382
6,Самарская область,940,3.33,5.0,66,254,56,14,28,522
7,Нижегородская область,830,3.07,4.0,56,283,49,10,25,407
8,Челябинская область,807,3.42,5.0,48,225,27,13,31,463
9,Москва и область,792,2.55,1.0,75,336,56,16,21,288


In [20]:
q = """
    select 
    distinct *, round(cast(status_cnt as numeric) / full_cnt * 100, 1) || ' %' as prcnt
    from (
        select 
            bank_name 
            , status 
            , count(*) over(partition by bank_name, status) as status_cnt
            , count(*) over(partition by bank_name) as full_cnt
        from 
            home.dt_banki_responses
        order by bank_name
    ) as t1
    order by bank_name
"""
custom_read_from_sql(q, 10)

,bank_name,status,status_cnt,full_cnt,prcnt
0,Альфа-Банк,Проблема решена,294,5203,5.7 %
1,Альфа-Банк,Без статуса,1567,5203,30.1 %
2,Альфа-Банк,Зачтено,786,5203,15.1 %
3,Альфа-Банк,Не засчитана,1651,5203,31.7 %
4,Альфа-Банк,Проверяется,905,5203,17.4 %
5,Банк Открытие,Проблема решена,1152,6207,18.6 %
6,Банк Открытие,Без статуса,1982,6207,31.9 %
7,Банк Открытие,Не засчитана,2118,6207,34.1 %
8,Банк Открытие,Проверяется,198,6207,3.2 %
9,Банк Открытие,Зачтено,757,6207,12.2 %
